In [ ]:
import tensorflow as tf
!wget -q http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar zxf aclImdb_v1.tar.gz
#!tree -d aclImdb

In [ ]:
NUM_WORDS=10000
SEQ_LEN=500
EMBEDDING_SIZE=128
BATCH_SIZE=128
EPOCHS=5
THRESHOLD=0.5

In [ ]:
import pandas as pd
import glob
import os
import string

def get_dfs(start_path):

  df = pd.DataFrame(columns=['text', 'sent'])
  text = []
  sent = []
  for p in ['pos','neg']:
    path=os.path.join(start_path, p)
    files = [f for f in os.listdir(path)
             if os.path.isfile(os.path.join(path,f))]
    for f in files:
      with open (os.path.join(path, f), "r") as myfile:
        # replace carriage return linefeed with spaces
        text.append(myfile.read()
                    .replace("\n", " ")
                    .replace("\r", " "))
        # convert positive reviews to 1 and negative reviews to zero
        sent.append(1 if p == 'pos' else 0)

  df['text']=text
  df['sent']=sent
  #This line shuffles the data so you don't end up with contiguous
  #blocks of positive and negative reviews
  df = df.sample(frac=1).reset_index(drop=True)      
  return df

train_df = get_dfs ("aclImdb/train/")
test_df = get_dfs ("aclImdb/test/")

train_df.head()


In [ ]:
#create tokenizer for our data
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS, oov_token='<UNK>')
tokenizer.fit_on_texts(train_df['text'])


#convert text data to numerical indexes
train_seqs=tokenizer.texts_to_sequences(train_df['text'])
test_seqs=tokenizer.texts_to_sequences(test_df['text'])

#pad data up to SEQ_LEN (note that we truncate if there are more than SEQ_LEN tokens)
train_seqs=tf.keras.preprocessing.sequence.pad_sequences(train_seqs, maxlen=SEQ_LEN, padding="post")
test_seqs=tf.keras.preprocessing.sequence.pad_sequences(test_seqs, maxlen=SEQ_LEN, padding="post")

In [ ]:

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(NUM_WORDS, EMBEDDING_SIZE),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
  tf.keras.layers.Dense(1, activation='sigmoid')])

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_seqs, train_df['sent'].values, batch_size=BATCH_SIZE, epochs=5, validation_split=0.2)

model.evaluate(test_seqs, test_df['sent'].values)[1]

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove.6B.zip

In [ ]:
import numpy as np
# Load GloVe pretrained embeddings
embedding_dim = 100
word_index = tokenizer.word_index

embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Word vectors: %s' % len(embeddings_index))
print('Embedding size: %s'% embedding_dim)

embedding_matrix = np.zeros((NUM_WORDS, embedding_dim))
for word, i in word_index.items():
    if i < NUM_WORDS:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(NUM_WORDS, 100),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
  tf.keras.layers.Dense(1, activation='sigmoid')])

model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_seqs, train_df['sent'].values, batch_size=BATCH_SIZE, epochs=5, validation_split=0.2)

model.evaluate(test_seqs, test_df['sent'].values)[1]

In [ ]:
model1 = tf.keras.Sequential([
  tf.keras.layers.Embedding(NUM_WORDS, EMBEDDING_SIZE),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')])

model1.summary()

model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max')
callbacks=[es]
history = model1.fit(train_seqs, train_df['sent'].values, batch_size=BATCH_SIZE, epochs=5, validation_split=0.2)

model1.evaluate(test_seqs, test_df['sent'].values)[1]